#### The following code uses Keras to build CNN model, in combination of training and testing data augmentation and pesudo labeling.

- The code is partially adapted from https://www.kaggle.com/devm2024/keras-model-for-beginners-0-210-on-lb-eda-r-d


- The features we use are band_1, band_2 and (band_1+band_2)/2. Inc_angle is not used and no additional feature engineering are used while I'm pretty sure the result will benefit from further engineering the features. This and other information can be found from: https://earth.esa.int/c/document_library/get_file?folderId=409229&name=DLFE-5566.pdf (Credit to Jian Wang)


- The input is a 75x75x3 set of images. The output is a probability bwtween 0/1 where 1 is noted as an iceberg.


- The code is written in the following order:
    - Data engineering and image normalization
    - data augmentation for engineered training data (X_train can contain multiple images)
    - data augmentation for engineered testing data (X_test also can contain multiple images)
    - pesudo labeling
    

- This code will generate the following results in .csv format:
    - prediction result without data augmentation
    - prediction result with only training data augmentation
    - prediction result with both training and testing data augmentation
    - prediction result with pesudo labeling

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [2]:
df_train = pd.read_json('train.json')
df_test = pd.read_json('test.json')

### 1. Data engineering and image normalization

In [3]:
# define image normalization function

def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((a, b, c)))

    return np.array(imgs)

In [4]:
# get X_train, y_train
X_train = get_scaled_imgs(df_train)
y_train = df_train['is_iceberg']

# get X_test
X_test = get_scaled_imgs(df_test)

# split data into training set and validation set
X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, y_train, random_state = 16, train_size = 0.75)

### 2. Import Keras and define CNN model
Note: There are still lots of room to tune the CNN model to optimize the prediction results.

In [5]:
#Import Keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

Using TensorFlow backend.


Define CNN model. 

The dimension of the input is (75, 75, 3) which corresponds to band_1, band_2 and (band_1 + band_2)/2.

In [6]:
# define CNN model
def getModel():
    #Building the model
    gmodel=Sequential()
    #Conv Layer 1
    gmodel.add(Conv2D(32, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))  # change input dimension
    gmodel.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 2
    gmodel.add(Conv2D(64, kernel_size=(3, 3), activation='relu' ))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 3
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 4
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Flatten the data for upcoming dense layers
    gmodel.add(Flatten())

    #Dense Layers
    gmodel.add(Dense(256)) # can be doubled
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Dense Layer 2
    gmodel.add(Dense(128)) # can be doubled
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Sigmoid Layer
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))

    mypotim = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    gmodel.compile(loss='binary_crossentropy',
                  optimizer=mypotim,
                  metrics=['accuracy'])
    gmodel.summary()
    return gmodel

# define filepath and callbacks
def get_callbacks(filepath, patience):
    es = EarlyStopping('val_loss', patience = patience, mode = "min")
    msave = ModelCheckpoint(filepath, save_best_only = True)
    return [es, msave]
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath = file_path, patience = 5)

# define parameters
batch_size = 128
epochs = 30

### 3. Result without data augmentation

In [8]:
# train and test validation set without data augmentation
gmodel = getModel()

gmodel.load_weights(filepath = '.model_weights.hdf5')

gmodel.fit(X_train_cv, y_train_cv, epochs=epochs, batch_size=batch_size, verbose = 1, validation_split = 0) 

loss, acc = gmodel.evaluate(X_valid, y_valid, batch_size = batch_size, verbose = 1)
print('Result without data augmentation')
print('Validation Loss: ', loss)
print('Validation Accuracy: ', acc)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 73, 73, 32)        896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 128)       73856     
__________

### 4. Train data augmentation using Keras
Here we use Keras ImageDataGenerator for the data augmentation. There are many parameters to tune and after spending time optimizing the parameters, we noticed that shift(including width and height), rotation, zoom and flip of the images can improve the performance of the model, while shear, samplewise_center and samplewise_std_normalization will significantly decrease the prediction accuracy. Channel shift has negligible influence on the model performance. ZCA whitening can also decrease the performance of the model. The parameters we use below is merely one set of parameters on which we achieved the best results among all those investigated. Further efforts can be put into optimizing the data augmentation parameters in order to improve the performance of the model.

In [9]:
# data augmentation using Keras
datagen = ImageDataGenerator(zoom_range = 0.2,
                         horizontal_flip = True,
                         vertical_flip = True
                       )

"manually" train the training dataset. Here the batch_size is 128 and the epochs number is 30*4 = 120.

In [10]:
# "manually" train the training dataset 
from keras.utils import generic_utils

gmodel = getModel()
for e in range(epochs*4):
    print('Epoch', e)
    print('Training...')
    progbar = generic_utils.Progbar(X_train_cv.shape[0])
    batches = 0

    for x_batch, y_batch in datagen.flow(X_train_cv, y_train_cv, batch_size = batch_size, shuffle = True):
        loss,train_acc = gmodel.train_on_batch(x_batch, y_batch)
        batches += x_batch.shape[0]
        if batches > X_train_cv.shape[0]:
            break
        progbar.add(x_batch.shape[0], values=[('train loss', loss),('train acc', train_acc)])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 73, 73, 32)        896       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 17, 17, 64)        0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 15, 15, 128)       73856     
__________

In [11]:
# train and test validation set with train data augmentation
loss, acc = gmodel.evaluate(X_valid, y_valid, batch_size=batch_size, verbose = 1)
print('Result with data augmentation')
print('Validation Loss: ', loss)
print('Validation Accuracy: ', acc)

401/401 [==============================] - 2s 5ms/step
Result with data augmentation
Validation Loss:  0.197882395303
Validation Accuracy:  0.922693266833


In [12]:
# predict test without test data augmentation
predicted_test = gmodel.predict_proba(X_test)

8424/8424 [==============================] - 44s 5ms/step


In [13]:
submission = pd.DataFrame()
submission['id'] = df_test['id']
submission['is_iceberg'] = predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('result_train_aug.csv', index = False)

### 5. Test data augmentation
Here we try to make sure the test dataset is augmented the same way as the train dataset. The predicted result of each entry is the average value of results from its multiple augmented images.

In [14]:
test_datagen_0 = ImageDataGenerator(zoom_range = 0.2) 
test_datagen_1 = ImageDataGenerator(horizontal_flip = True) 
test_datagen_2 = ImageDataGenerator(vertical_flip = True) 

testbatch_size = 1

predicted_test = np.asarray([0.0]*8424,dtype = np.float32) # the test data has 8424 entries.
predicted_test = predicted_test.reshape(predicted_test.shape + (1,))
num_para = 3
j = 0

while j < num_para:
    X_test_aug = []
    for i in range(len(X_test)):
        k = 0
        X_test_reshape = X_test[i].reshape((1,) + X_test[i].shape) 
        i += 1
        for batch in eval('test_datagen_{}.flow(X_test_reshape, batch_size = testbatch_size, seed = 0)'.format(j)):      
            X_test_aug.append(batch.squeeze(axis = 0))
            k += 1
            if k >= testbatch_size:
                break
    j += 1 
    X_test_aug = np.asarray(X_test_aug, dtype=np.float32)
    result = gmodel.predict_proba(X_test_aug)
    
    # take the average
    predicted_test += np.asarray(result, dtype = np.float32)/(num_para*1.0)


8424/8424 [==============================] - 41s 5ms/step


In [15]:
submission = pd.DataFrame()
submission['id'] = df_test['id']
submission['is_iceberg'] = predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('result_train_test_aug.csv', index=False)

### 6. Pseudo labeling
Pseudo labeling is a semi-supervised machine learning method used when it's hard to get enough training dataset. The general procedure is 1) Use the model to predict the testing dataset 2) use the result as label to retrain the combination of both the training and testing dataset 3) Use the retrained model with psuedo labeling data to predict the test sample. Please refer to the following link for details:
https://www.analyticsvidhya.com/blog/2017/09/pseudo-labelling-semi-supervised-learning-technique/

1) Use the model to predict the testing dataset

In [16]:
pred_test_classes = gmodel.predict_classes(X_test)

8424/8424 [==============================] - 43s 5ms/step


2) Retrain the combination of both the training and testing dataset

In [17]:
X_train_total = np.concatenate((X_test,X_train),axis = 0)
y_train_total = np.concatenate((pred_test_classes,y_train[:,np.newaxis]),axis = 0)

In [19]:
gmodel.fit(X_train_total, y_train_total, batch_size=batch_size, epochs=50, verbose=1, validation_split = 0)

Epoch 1/50
10028/10028 [==============================] - 168s 17ms/step - loss: 0.1956 - acc: 0.9181
Epoch 2/50
10028/10028 [==============================] - 165s 16ms/step - loss: 0.1709 - acc: 0.9305
Epoch 3/50
10028/10028 [==============================] - 168s 17ms/step - loss: 0.1625 - acc: 0.9317
Epoch 4/50
10028/10028 [==============================] - 156s 16ms/step - loss: 0.1508 - acc: 0.9389
Epoch 5/50
10028/10028 [==============================] - 156s 16ms/step - loss: 0.1398 - acc: 0.9419
Epoch 6/50
10028/10028 [==============================] - 158s 16ms/step - loss: 0.1317 - acc: 0.9457
Epoch 7/50
10028/10028 [==============================] - 158s 16ms/step - loss: 0.1317 - acc: 0.9455
Epoch 8/50
10028/10028 [==============================] - 162s 16ms/step - loss: 0.1286 - acc: 0.9453
Epoch 9/50
10028/10028 [==============================] - 167s 17ms/step - loss: 0.1124 - acc: 0.9568
Epoch 10/50
10028/10028 [==============================] - 160s 16ms/step - loss: 

3) Use the retrained model with psuedo labeling data to predict the test sample

In [20]:
loss, acc = gmodel.evaluate(X_train_total, y_train_total, batch_size=batch_size, verbose = 1)
print('Result with pseudo labeling')
print('Validation Loss: ', loss)
print('Validation Accuracy: ', acc)

10028/10028 [==============================] - 49s 5ms/step
Result with pseudo labeling
Validation Loss:  0.0019593254536
Validation Accuracy:  0.999900279218


In [21]:
predicted_test = gmodel.predict_proba(X_test)

8424/8424 [==============================] - 44s 5ms/step


In [22]:
submission = pd.DataFrame()
submission['id'] = df_test['id']
submission['is_iceberg'] = predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('result_pseudo_labeling.csv', index = False)

### Summary

1. Data normalization significantly improves the accuracy of the prediction results (log loss decrease from ~0.21 to ~0.19).
2. Both train data augmentation and test data augmentation improves the accuracy of the prediction results (log loss decrease ~ 0.01).
3. Pesudo-labeling is tricky here. 
   - By definition, we can tell that it can push the prediction results to extremes, either much better or much less. 
   - In this case, the training dataset is only ~ 25% of the testing dataset. It is quite likely that the prediction error
     on the testing dataset is propagated into the second round of training and further boosted. Therefore, the log loss      increased to as high as ~ 0.78. 
   - One key parameter to tune here is retrain the augmented training dataset and testing dataset, where much more data
are included and overfitting can probably better avoided.